### Importing necessary libraries

In [8]:
pip install scapy

Defaulting to user installation because normal site-packages is not writeable
  Using cached scapy-2.6.1-py3-none-any.whl.metadata (5.6 kB)
Using cached scapy-2.6.1-py3-none-any.whl (2.4 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\pavan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
import scapy.all as scapy
import pandas as pd
import joblib  # For loading the trained model
from datetime import datetime
import logging
from sklearn.preprocessing import LabelEncoder
import time

In [21]:
import scapy.all as scapy
import pandas as pd
import joblib  # For loading the trained model
from datetime import datetime
import logging
from sklearn.preprocessing import LabelEncoder
import time

# Configure logging
logging.basicConfig(filename="nids_log.txt", level=logging.INFO, format="%(asctime)s - %(message)s")

# Load the trained model
model = joblib.load("nids_model.pkl")  # Ensure 'nids_model.pkl' is available

# Store active flows
active_flows = {}

# Timeout to end an active flow (in seconds)
FLOW_TIMEOUT = 60  # If no packet arrives within this time, the flow is considered ended

# Define feature extraction function based on your dataset structure
def extract_features(packet):
    features = {}
    
    # Extract features related to IPV4 and ports
    if packet.haslayer(scapy.IP):
        features['IPV4_SRC_ADDR'] = packet[scapy.IP].src
        features['IPV4_DST_ADDR'] = packet[scapy.IP].dst
        features['PROTOCOL'] = packet[scapy.IP].proto
        features['L4_SRC_PORT'] = packet.sport if packet.haslayer(scapy.TCP) else 0
        features['L4_DST_PORT'] = packet.dport if packet.haslayer(scapy.TCP) else 0
        
        # Extract TCP flags
        features['TCP_FLAGS'] = packet.sprintf("%TCP.flags%") if packet.haslayer(scapy.TCP) else "None"
        
        # Application Layer Protocol (Layer 7 protocol)
        features['L7_PROTO'] = packet.payload.__class__.__name__ if hasattr(packet.payload, '__class__') else "Unknown"
        
        # Bytes and packets count (in and out)
        features['IN_BYTES'] = len(packet)
        features['OUT_BYTES'] = 0  # You can extend this if you want to track outgoing packets
        features['IN_PKTS'] = 1  # We assume 1 packet at a time
        features['OUT_PKTS'] = 0  # Could track outbound if required

        # Flow Duration (could be zero for single packets or calculated differently for flows)
        features['FLOW_DURATION_MILLISECONDS'] = 0  # Flow duration tracking can be extended if needed
    
    return features

# Update active flow information
def update_flow_info(packet, flow_key, features):
    if flow_key not in active_flows:
        # Start a new flow
        active_flows[flow_key] = {
            'start_time': datetime.now(),
            'last_packet_time': datetime.now(),
            'total_bytes': 0,
            'total_packets': 0,
            'tcp_flags': set(),
            'protocol': features['PROTOCOL'],
            'src_ip': features['IPV4_SRC_ADDR'],
            'dst_ip': features['IPV4_DST_ADDR'],
            'src_port': features['L4_SRC_PORT'],
            'dst_port': features['L4_DST_PORT']
        }
    
    # Update flow statistics
    flow = active_flows[flow_key]
    flow['last_packet_time'] = datetime.now()
    flow['total_bytes'] += features['IN_BYTES']
    flow['total_packets'] += 1
    flow['tcp_flags'].add(features['TCP_FLAGS'])

    # Calculate flow duration in milliseconds
    flow['FLOW_DURATION_MILLISECONDS'] = (flow['last_packet_time'] - flow['start_time']).total_seconds() * 1000
    
def cleanup_old_flows():
    # Clean up flows that have timed out
    current_time = datetime.now()
    for flow_key, flow in list(active_flows.items()):
        time_diff = (current_time - flow['last_packet_time']).total_seconds()
        if time_diff > FLOW_TIMEOUT:
            print(f"Flow {flow_key} timed out.")
            del active_flows[flow_key]

# Process packet and make prediction
def process_packet(packet):
    features = extract_features(packet)
    if features:
        # Create a flow key (a unique identifier for each flow)
        flow_key = (features['IPV4_SRC_ADDR'], features['IPV4_DST_ADDR'], features['L4_SRC_PORT'], features['L4_DST_PORT'], features['PROTOCOL'])

        # Update flow information
        update_flow_info(packet, flow_key, features)

        # Convert flow info to DataFrame for prediction
        flow = active_flows[flow_key]
        flow_data = {
            'PROTOCOL': flow['protocol'],
            'L4_SRC_PORT': flow['src_port'],
            'L4_DST_PORT': flow['dst_port'],
            'TCP_FLAGS': ','.join(flow['tcp_flags']),
            'IN_BYTES': flow['total_bytes'],
            'IN_PKTS': flow['total_packets'],
            'FLOW_DURATION_MILLISECONDS': flow['FLOW_DURATION_MILLISECONDS']
        }

        df = pd.DataFrame([flow_data])

        # Encode categorical features
        categorical_features = ['PROTOCOL', 'TCP_FLAGS']
        label_encoder = LabelEncoder()
        
        # Label encoding for categorical features
        for feature in categorical_features:
            if feature in df.columns:
                df[feature] = label_encoder.fit_transform(df[feature].values)
        
        try:
            # Make prediction using the model
            prediction = model.predict(df)[0]
            label = "Malicious" if prediction == 1 else "Benign"

            # Log detection
            log_message = f"Detected {label} traffic from {features['IPV4_SRC_ADDR']} to {features['IPV4_DST_ADDR']} with protocol {features['PROTOCOL']}"
            logging.info(log_message)
            print(f"[ALERT] {log_message}")

            # Automated Response: Example - Block malicious IP
            if label == "Malicious":
                block_ip(features['IPV4_SRC_ADDR'])

        except Exception as e:
            logging.error(f"Error during prediction: {e}")
            print(f"Error during prediction: {e}")

# Function to block IP (Linux IPTables example)
def block_ip(ip):
    print(f"Blocking IP: {ip}")
    # Uncomment below to actually block
    # os.system(f"sudo iptables -A INPUT -s {ip} -j DROP")
    logging.info(f"Blocked IP: {ip}")

# Clean up old flows periodically (for example, every 30 seconds)
import threading
def periodic_cleanup():
    while True:
        time.sleep(FLOW_TIMEOUT)  # Run cleanup periodically based on the flow timeout
        cleanup_old_flows()

# Start a cleanup thread
cleanup_thread = threading.Thread(target=periodic_cleanup, daemon=True)
cleanup_thread.start()

# Start sniffing network traffic
print("Starting real-time Network Intrusion Detection System (NIDS)...")
scapy.sniff(prn=process_packet, store=False)


Starting real-time Network Intrusion Detection System (NIDS)...


RuntimeError: Sniffing and sending packets is not available at layer 2: winpcap is not installed. You may use conf.L3socket or conf.L3socket6 to access layer 3

In [19]:
from scapy import *
# Packet sniffing example
def packet_callback(packet):
    print(packet.show())

sniff(prn=packet_callback, count=10)

# Packet sending example
packet = IP(dst="8.8.8.8")/ICMP()
send(packet)


RuntimeError: Sniffing and sending packets is not available at layer 2: winpcap is not installed. You may use conf.L3socket or conf.L3socket6 to access layer 3

In [15]:
# Install Scapy
!pip install scapy

# Import Scapy
from scapy.all import *

# Packet sniffing example
def packet_callback(packet):
    print(packet.show())

sniff(prn=packet_callback, count=10)

# Packet sending example
packet = IP(dst="8.8.8.8")/ICMP()
send(packet)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0

[notice] To update, run: C:\Users\pavan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

RuntimeError: Sniffing and sending packets is not available at layer 2: winpcap is not installed. You may use conf.L3socket or conf.L3socket6 to access layer 3

In [ ]:
import streamlit as st
import pandas as pd
import time

# Define the log file path
LOG_FILE = "nids_log.txt"

# Streamlit dashboard setup
st.title("Real-Time Network Intrusion Detection System (NIDS) Dashboard")
st.write("Monitoring real-time network traffic for intrusion detection.")

# Function to read logs
def read_logs():
    try:
        df = pd.read_csv(LOG_FILE, sep=" - ", engine="python", names=["Timestamp", "Message"])
        df["Timestamp"] = pd.to_datetime(df["Timestamp"])
        return df
    except Exception as e:
        return pd.DataFrame(columns=["Timestamp", "Message"])

# Real-time updates
placeholder = st.empty()
while True:
    df = read_logs()
    if not df.empty:
        with placeholder.container():
            st.dataframe(df.tail(10))  # Show last 10 alerts
    time.sleep(5)  # Refresh every 5 seconds
